# Final Project - Modeling and forecasting atmospheric CO$_2$ from 1958 into the future

Carbon levels in Earth's atmosphere has been fluctuating a lot over thousands of years that the earth has been existant.

![Carbon levels](carbon.jpg)

**Figure 1:** Atmospheric carbon levels in the past 800,000 years. (Climate Change: Atmospheric Carbon Dioxide, n.d.)

In spite of the fluctuations, current levels of atmospheric carbon are unprecedented. The recent increase in carbon levels may be attributed to human activities especially after the industrial age. Given that the Earth has never experienced such levels before, nature might not have adapted to such levels. Could the recent mass extinction of species be linked to increasing levels of carbon? We will not be looking into the causality or effects of increasing carbon levels; however, they may be very interesting. Here we are going to develop a model that would help us forecast future levels of atmospheric carbon given data from the past several years. Such a model would help us identify if we deviate from the general trend predicted positively or negatively. It also helps us identify, when CO$_2$ levels would likely reach dangerous levels such as 450 ppm.

## The Data

The data used is from Mauna Loa Observatory and spans from 1958 to the present with recordings occurring every week. The data contains the level of carbon in PPM recorded as well as the date of recording. To build the model, we add an extra feature that represents the number of days since 1958 January 1. We do this because it is easier modelling numbers than date datatypes.

Here is a look at the trend of the data from 1958, to a recent time.

![Carbon levels recently](observed_data.png)

**Figure 2:** Carbon level recordings from the Manua Loa observatory.

From the plot we may notice that the levels follow a zigzag motion. That may be attributed to changing seasons annually on Earth. During some seasons, such as Spring and Summer, there is more life in terms of flourishing plants that absorb CO$_2$; while during the winter, decidious trees shed their leaves meaning their carbon intake levels go down leading to an increase in the atmosphere.

## Modelling

### Bad Model

In the assignment instructions, we were provided with a model of atmospheric carbon levels; however, it is a bad model. The bad model invovles:
* Long-term trend: linear, $c_{0} + c_{1} t$
* Seasonal variation (every 3651⁄4 days): cosine, $c_{2} \cos(2 \pi t / 365.25 + c_{3})$
* Noise: Gaussian with mean 0 and fixed standard deviation, $c_{4}$
* The $c_{i}$ variables are all unobserved parameters of the model.

Combining these three components gives the following likelihood function
$$p(x_{t} | \theta) = N(c_{0} +c_{1} t + c_{2} \cos(2 \pi t / 365.25 + c_{3}), c^{2}_{4})$$
where $\theta$ represents the set of all unobserved parameters.

Building such a model makes as quite apparent with its flaws.

![Bad model](bad_model.png)

**Figure 3:** Modelling CO$_2$ levels using a bad model.

The most obvious reason of why it is a bad model, is that the data is not linear. A more appropriate trend is polynomial or exponential.

### Good Model

To resolve the main flaw of the previous model, we use a polynomial trend to build a better model. For the seasonal variation, we maintain the same cosine function since it seems to model the seasonal variation quite accurately. Similarly, for the noise, we maintain a Gaussian with mean 0 and fixed standard deviation.

The good model has the following likelihood:
$$p(x_{t} | \theta) = N(\text{intercept} +a \cdot t + b \cdot t^{2} + c_{2} \cos(2 \pi t / 365.25 + c_{3}), c^{2}_{4})$$

Here is a Directed Acrylic Graph(DAG) representing the good model.

![Directed Acrylic Graph](dag.jpg)

**Figure 4:** A DAG representing the good model.

The observed variables are the $x$ values which are the recordings of levels of CO$_2$ in parts per million (ppm). While the unobserved variables are the parameters to the model that yield the observed value.

#### Priors

As part of prior information, we know that in 1958, the levels of CO_$2$, were above 0, y_intercept, of the model would have to have a lower limit of 0.

The $a$ and $b$ parameters also have a lower limit of 0, because as we saw the observed data seems to have an upward trend, therefore, we expect these coefficients are positive representing an upward trend.

$c_{2}$, is also positive, since it represents the amplitude of the cosine function. From the seasonal, variation, we can see that it should be positive with a visible amplitude.

$sigma$, is also given as positive, because standard deviation is always a positive number in normal scenarios. 

Finally $c_{3}$, is limited within the range of -2, 2 based on running the model multiple times as well as the results from the bad model, hinted us to limit it within this range to have good sampling results in the end.

Here is the stan code representing these limits.
```c++
parameters {
    //posterior parameters
    real<lower=0> intercept;
    real<lower=0> a;
    real<lower=0> b;
    real<lower=0> c2;
    real<lower=-2, upper=2> c3;
    real<lower=0> sigma;
}
```

We used prior distributions for some of the variables. 
* For the intercept, we use a cauchy distribution centred at 300, and a deviation of 50.
* For the $b$ coefficitient, a cauchy centred at 0, and a deviation of 5.
* $c_{2}$, cauchy centred at 2.5, and a deviation of 1.
* $c_{3}$, cauchy centred at 0, and a deviation of 1. 

For the ones centred at 0, but have a lower limit of 0, then they have a truncated cauchy. We use a cauchy distribution because of its wider tails and a higher probability of sampling at the tails than a normal distribution.

Here is the Stan code that represents these priors.

```c++
intercept ~cauchy(300, 50);
b ~cauchy(0, 5);
c2 ~ cauchy(2.5, 1);
c3 ~ cauchy(0, 1);
```

## Results - Posterior

Using Stan, we were able to estimate the parameters of the model. Here are the results we got.

```
            mean se_mean      sd   2.5%    25%    50%     75%  97.5%  n_eff   Rhat
intercept 314.54  1.3e-3    0.05 314.43  314.5 314.54  314.57 314.65   1777    1.0
a         2.0e-3  2.9e-7  1.1e-5 2.0e-3 2.0e-3 2.0e-3  2.0e-3 2.1e-3   1335    1.0
b         9.9e-8 1.2e-11 4.4e-10 9.9e-8 9.9e-8 9.9e-8 10.0e-8 1.0e-7   1314    1.0
c2          2.86  4.8e-4    0.02   2.81   2.84   2.86    2.87   2.91   2701    1.0
c3         -1.91  1.6e-4  8.7e-3  -1.92  -1.91  -1.91    -1.9  -1.89   3031    1.0
sigma       0.98  2.1e-4    0.01   0.96   0.98   0.98    0.99   1.01   3185    1.0
```

We notice that the intercept, indicating that on day 0, which is 1958, January 1, the CO$_2$ level was 314.54 ppm.
$a$, and $b$ respectively indicate that every day, the level of CO$_2$ increases by 2.0e-3 and 9.9e-8. 

$c_2$ represents the annual variation through seasons, and $c_{3}$ is an adjustment figure inside the seasonality cosine function. 

The noise standard deviation is very low in this case at 0.98.

Plotting, the good model together with the observed data we get:

![Good model](good_model.png)

**Figure 5:** A good model that models the CO$_2$ levels.

This model seems to do a good job, because we can barely see the blue line of observed data that is under the orange line representing the model.

We expect the confidence intervals of the future data to widen the further in the future the prediction is for, however, we don't get that. We only see a very tight 95% confidence interval. 

Based on the results from stan, the `rhat` values of all the parameters are 1 (or rather very close to 1) meaning that the different markov chains mixed well leading to similar results in them. Next, we have a `n_eff` values greater than 1000 for all of them representing enough effective samples.

Additional checks include a pairplot of all the parameters. If we got good sampling, then the parameter pair plots would show random scatters between the pairs. In our case, that is what we get.

![Pair plot](pair_plot.png)


We can also check the autocorrelation of each parameters samples. We expect a value 1 for 0 on the x-axis, then values close to 0 for the rest of the values on the x-axis symbolysing very low auto-correlation between samples.

![Auto-correlation intercept](corr_intercept.png)
![Auto-correlation a](corr_a.png)
![Auto-correlation b](corr_b.png)
![Auto-correlation c2](corr_c2.png)
![Auto-correlation c3](corr_c3.png)
![Auto-correlation sigma](corr_sigma.png)

From the plot of the model, and a quick quantitative calculation from the future confidence intervals, it seems we will definitely surpass 450 ppm on February 10, 2035. This would be a milestone because that is the level that scientists are warning us of (Oecd & OECD, 2012). It may lead to an incrase of global average temperatures by 2 degrees. It seems a little but when it is global then it might have dire consequences.

Given that Earth is a complex system, it may undergo reinforcing feedback loops. For example, as the temperature warms due to increasing CO$_2$ levels, then more polar ice would melt releasing ancient dissolved CO$_2$. As the same ice melts, the albedo or reflectivity of the earth surface would reduce, meaning there would be higher solar insolation leading to warmer temperatures still. 

Many big companies, are planning on doing major changes by 2030, such as being carbon neutral or carbon negative such as Google, Apple and Microsoft, however, a much bigger change would be a push from the governements and a global cooperation. After World War 2, there were multiple global efforts that worked such as the ending of small pox through effective immunisation programs across the globe, however, the recent 2020 pandemic didn't experience a similar level of global cooperation. What would be the case for the climate problem? Only time would tell. However, it is up to all of us to play our part and reduce not only carbon emmissions but also making changes that would improve the environment around us instead of hampering it.

## References

Climate Change: Atmospheric Carbon Dioxide. (n.d.). Retrieved April 29, 2021, from [https://www.climate.gov/news-features/understanding-climate/climate-change-atmospheric-carbon-dioxide](https://www.climate.gov/news-features/understanding-climate/climate-change-atmospheric-carbon-dioxide)

Oecd, & OECD. (2012). OECD Environmental Outlook to 2050. In OECD Environmental Outlook. [https://doi.org/10.1787/9789264122246-en](https://doi.org/10.1787/9789264122246-en)
